In [1]:
import pandas as pd
import numpy as np
import os
import sys


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

In [2]:
def feature_dataframe_gen(feature_dir):
    feature_dataframe = []
    for sess in range(1,6) :
        feature_df = pd.read_csv('{}audio_features_{}.csv'.format(feature_dir,sess))
        feature_df = feature_df.sample(frac=1,random_state = 50).reset_index(drop = True)
        feature_dataframe.append(feature_df)
    feature_dataframe = (pd.concat(feature_dataframe)).fillna(0)
    feature_dataframe = feature_dataframe.replace('exc','hap')
    #feature_dataframe = feature_dataframe.drop(feature_dataframe[feature_dataframe.emotions.isin(["sur", "fea"])].index)
    return feature_dataframe

In [3]:
ori_feature_dataframe = feature_dataframe_gen(feature_dir = '../input/iemocap-audio-features/')

In [4]:
spec_feature_dataframe = feature_dataframe_gen(feature_dir = '../input/iemocap-specaugmented/')

In [5]:
ori_feature_dataframe = ori_feature_dataframe.loc[:, (ori_feature_dataframe==0.0).mean() < .9]

In [6]:
spec_feature_dataframe = spec_feature_dataframe.loc[:, (spec_feature_dataframe==0.0).mean() < .9]

In [7]:
X_train_ori, X_test_ori, y_train_ori, y_test_ori = train_test_split(ori_feature_dataframe.drop('emotions',1), ori_feature_dataframe.emotions, test_size = 0.20, shuffle = True, random_state = 42)

In [8]:
X_train_spec, X_test_spec, y_train_spec, y_test_spec = train_test_split(spec_feature_dataframe.drop('emotions',1), spec_feature_dataframe.emotions, test_size = 0.20, shuffle = True, random_state = 42)

In [9]:

#X_train = X_train_ori
#y_train = y_train_ori
X_train = pd.concat([X_train_ori,X_train_spec]).fillna(0)
y_train = pd.concat([y_train_ori,y_train_spec])
X_test = X_test_ori
y_test = y_test_ori


mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = ((X_train - mean)/std)
X_test = ((X_test - mean)/std)

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

X_train = np.expand_dims(X_train, axis = 2)
X_test = np.expand_dims(X_test, axis = 2)

In [11]:

model = Sequential()

model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.05))
model.add(MaxPooling1D(pool_size=(8)))

model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.10))
model.add(MaxPooling1D(pool_size=(8)))

model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4)) # Target class number
model.add(Activation('softmax'))
opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
#opt = keras.optimizers.Adam(lr=0.000001)
#opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2994, 256)         2304      
_________________________________________________________________
activation (Activation)      (None, 2994, 256)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2994, 256)         524544    
_________________________________________________________________
batch_normalization (BatchNo (None, 2994, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 2994, 256)         0         
_________________________________________________________________
dropout (Dropout)            (None, 2994, 256)         0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 374, 256)          0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),'accuracy'])
history=model.fit(X_train, y_train, batch_size=16, epochs=60, validation_data=(X_test, y_test))

hist_df = pd.DataFrame(history.history) 
hist_df.to_csv('./single_iemocap.csv')

Epoch 1/60
553/553 [==============================] - 22s 40ms/step - loss: 1.3407 - recall: 0.0460 - precision: 0.4834 - accuracy: 0.3562 - val_loss: 1.3320 - val_recall: 0.0136 - val_precision: 0.4167 - val_accuracy: 0.4074
Epoch 2/60
553/553 [==============================] - 21s 38ms/step - loss: 1.2781 - recall: 0.0568 - precision: 0.5503 - accuracy: 0.4053 - val_loss: 1.3032 - val_recall: 0.0280 - val_precision: 0.4769 - val_accuracy: 0.4137
Epoch 3/60
553/553 [==============================] - 21s 38ms/step - loss: 1.2574 - recall: 0.0597 - precision: 0.5796 - accuracy: 0.4081 - val_loss: 1.2932 - val_recall: 0.0271 - val_precision: 0.4615 - val_accuracy: 0.4065
Epoch 4/60
553/553 [==============================] - 21s 39ms/step - loss: 1.2377 - recall: 0.0650 - precision: 0.6002 - accuracy: 0.4173 - val_loss: 1.2827 - val_recall: 0.0325 - val_precision: 0.4865 - val_accuracy: 0.4146
Epoch 5/60
553/553 [==============================] - 21s 38ms/step - loss: 1.2198 - recall: 0.0

In [ ]:
model_name = 'Emotion_recognition_iemocap_specAugment.h5'
save_dir = os.path.join(os.getcwd(),'saved_models')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
model_path = os.path.join(save_dir,model_name)
model.save(model_path)
print("save model and weight at %s" %model_path)
#save model to disk
model_json = model.to_json()
with open("model_json.json", "w") as file:
    file.write(model_json)

In [ ]:
# loading json and model architecture 
json_file = open('./model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("./saved_models/Emotion_recognition_iemocap_specAugment.h5")
print("Loaded model from disk")
 
# Keras optimiser
opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

preds = loaded_model.predict(X_test, batch_size=16, verbose=1)

preds=preds.argmax(axis=1)

print(preds)

In [ ]:
preds = preds.astype(int).flatten()
preds = (lb.inverse_transform((preds)))
preds = pd.DataFrame({'predictedvalues': preds})

# Actual labels
actual=y_test.argmax(axis=1)
actual = actual.astype(int).flatten()
actual = (lb.inverse_transform((actual)))
actual = pd.DataFrame({'actualvalues': actual})

# Lets combined both of them into a single dataframe
finaldf = actual.join(preds)

# Write out the predictions to disk
finaldf.to_csv('Predictions.csv', index=False)
print(finaldf.groupby('predictedvalues').count())

In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
finaldf = pd.read_csv("./Predictions.csv")
classes = finaldf.actualvalues.unique()
classes.sort()    
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
# Confusion matrix 
c = confusion_matrix(finaldf.actualvalues, finaldf.predictedvalues)
print(accuracy_score(finaldf.actualvalues, finaldf.predictedvalues))
plt.show(print_confusion_matrix(c, class_names = classes))


In [ ]:
# Classification report 
classes = finaldf.actualvalues.unique()
classes.sort()    
print(classification_report(finaldf.actualvalues, finaldf.predictedvalues, target_names=classes))

In [ ]:
history_df = pd.read_csv('./specAugment_iemocap.csv')
history_df

In [ ]:
plt.plot(history_df.accuracy)
plt.plot(history_df.val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_df.loss)
plt.plot(history_df.val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()